# Entrenamiento del modelo

Importacion de librerias

In [1]:
import cv2
import glob
import os
import random
import string
import logging
import time

import numpy as np
from keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

#librerias para el modelo
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.optimizers import SGD
from keras import backend as K

Definicion del modelo

In [2]:
class NetworkModel:
    def build(width, height, depth, out_size):
        # crea el modelo
        model = Sequential()
        inputShape = (height, width, depth)

        # verifica el formato de la imagen
        # (dependente de backend)
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)

        
        model.add(Conv2D(32, (3, 3), padding='valid', input_shape=inputShape))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(2))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(2))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(2))
        model.add(Dropout(0.25))

        # Fully connected layer
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.25))
        model.add(Dense(out_size))
        model.add(Activation('softmax'))
        
        return model

Definicion de funciones y recoleccion de datos para el entrenamiento

In [3]:
# fijamos una semilla para mantener los resultados producibles
random.seed(71)

#declaracion alfabeto
alphabet = string.digits + string.ascii_letters
#Declaracion del alfabeto reconozible por el programa
recognizer_alphabet = ''.join(sorted(set(alphabet.upper())))

#Tranforma un caracter a represetacion one-hot
def encode(char):
    arr = np.zeros((len(recognizer_alphabet),), dtype="uint8")
    index = recognizer_alphabet.index(char)
    arr[index] = 1
    return arr

#carga los datos y etiquetas en dow vectores
#num limita la cantidad de muestras
def load_dataset(num):
    #ruta de datos
    seg_path = "../dataset/data_train/clasification/"
    
    #listas de datos y etiquetas
    data = []
    labels = []
    
    for char in recognizer_alphabet:
        print(f"Cargando datos del caracter '{char}'")

        #actualizacion de rutas
        path = seg_path + char + "/"
        files = os.listdir(path)

        for file in files[:num]:
            #opbtencion de la imagen y reescalado
            image = cv2.imread(path + file)
            resized = cv2.resize(image, (30, 30))

            #etiquetado one-hot
            label = encode(char)

            #guardadno datos
            data.append(resized)
            labels.append(label)                
    return data, labels

#normaliza las imagenes de [0-255] para [0-1] 
#convierte los datos en formato numpy
def normalize_samples(data, labels):
    n_data = np.array(data, dtype="float") / 255.0
    n_labels = np.array(labels)
    return n_data, n_labels

#entrena la red neuronal salvando su historico como num.csv
def train_network(train_x, train_y, validation_x, validation_y, epochs, learning_rate, batch_size, min_delta, patience, model_name):
    #ruta para guardar el modelo o resultados
    mod_path = "../models/" + model_name + "/"

    if not os.path.isdir(mod_path):
        os.makedirs(mod_path)

    #optimizacion: descenso de gradiente estocástico
    sgd = SGD(lr=learning_rate)

    #Se compila del modelo de red neuronal
    model = NetworkModel.build(30, 30, 3, len(recognizer_alphabet))
    model.compile(loss='categorical_crossentropy', 
                optimizer=sgd, 
                metrics=['accuracy'])

    csv_logger = CSVLogger(mod_path + f'results.csv', separator=";")
    # callback de detención anticipada
    early_stop = EarlyStopping(monitor='val_loss', mode='auto', min_delta=min_delta, patience=patience)

    #entrenamiento
    model.fit(train_x, train_y, 
                validation_data=(validation_x, validation_y), 
                batch_size=batch_size, 
                epochs=epochs, 
                verbose=1,
                callbacks=[early_stop, csv_logger])

    #guarda el modelo
    model.save(mod_path + f'model.hdf5')

if __name__ == "__main__":
    # min_delta = 1e-6, 1e-5, 1e-4
    # paciencia = 0, 5, 10
    num = 4000
    epochs = 256
    learning_rate = 1e-3
    batch_size = 128
    validation_split=0.66
    min_delta = 1e-6
    patience = 10


    print("Cargando dataset...")
    data, labels = load_dataset(num)

    print("Normalizando muestras...")
    n_data, n_labels = normalize_samples(data, labels)

    print("Separando el entrenamiento y la verificacion...")
    (train_x, validation_x, train_y, validation_y) = train_test_split(n_data, n_labels, test_size=0.3, random_state=42)

    print("entrenando red...")
    # nombre de archivo que marca el tiempo  
    timestr = time.strftime("%Y%m%d-%H%M%S")
    model_name = f"model-md[{str(min_delta)}]-pt[{str(patience)}]"
    print(f"Nombre del modelo: {model_name}")
    
    train_network(train_x, train_y, validation_x, validation_y, epochs, learning_rate, batch_size, min_delta, patience, model_name)

    print("Entrenamiento concluido!")
    print(f"Los resultados fueron guardados en '../models/{model_name}'")

Cargando dataset...
Cargando datos del caracter '0'
Cargando datos del caracter '1'
Cargando datos del caracter '2'
Cargando datos del caracter '3'
Cargando datos del caracter '4'
Cargando datos del caracter '5'
Cargando datos del caracter '6'
Cargando datos del caracter '7'
Cargando datos del caracter '8'
Cargando datos del caracter '9'
Cargando datos del caracter 'A'
Cargando datos del caracter 'B'
Cargando datos del caracter 'C'
Cargando datos del caracter 'D'
Cargando datos del caracter 'E'
Cargando datos del caracter 'F'
Cargando datos del caracter 'G'
Cargando datos del caracter 'H'
Cargando datos del caracter 'I'
Cargando datos del caracter 'J'
Cargando datos del caracter 'K'
Cargando datos del caracter 'L'
Cargando datos del caracter 'M'
Cargando datos del caracter 'N'
Cargando datos del caracter 'O'
Cargando datos del caracter 'P'
Cargando datos del caracter 'Q'
Cargando datos del caracter 'R'
Cargando datos del caracter 'S'
Cargando datos del caracter 'T'
Cargando datos del c

Epoch 49/256
781/781 [==============================] - 5s 7ms/step - loss: 0.5005 - accuracy: 0.8510 - val_loss: 0.3402 - val_accuracy: 0.9054
Epoch 50/256
781/781 [==============================] - 5s 7ms/step - loss: 0.4884 - accuracy: 0.8549 - val_loss: 0.3361 - val_accuracy: 0.9069
Epoch 51/256
781/781 [==============================] - 5s 7ms/step - loss: 0.4892 - accuracy: 0.8540 - val_loss: 0.3315 - val_accuracy: 0.9072
Epoch 52/256
781/781 [==============================] - 5s 7ms/step - loss: 0.4833 - accuracy: 0.8551 - val_loss: 0.3270 - val_accuracy: 0.9088
Epoch 53/256
781/781 [==============================] - 5s 7ms/step - loss: 0.4754 - accuracy: 0.8592 - val_loss: 0.3232 - val_accuracy: 0.9094
Epoch 54/256
781/781 [==============================] - 6s 7ms/step - loss: 0.4648 - accuracy: 0.8613 - val_loss: 0.3202 - val_accuracy: 0.9103
Epoch 55/256
781/781 [==============================] - 5s 7ms/step - loss: 0.4643 - accuracy: 0.8620 - val_loss: 0.3153 - val_accuracy:

Epoch 106/256
781/781 [==============================] - 5s 7ms/step - loss: 0.3124 - accuracy: 0.9038 - val_loss: 0.2168 - val_accuracy: 0.9336
Epoch 107/256
781/781 [==============================] - 5s 7ms/step - loss: 0.3113 - accuracy: 0.9027 - val_loss: 0.2151 - val_accuracy: 0.9337
Epoch 108/256
781/781 [==============================] - 5s 7ms/step - loss: 0.3149 - accuracy: 0.9028 - val_loss: 0.2141 - val_accuracy: 0.9341
Epoch 109/256
781/781 [==============================] - 5s 7ms/step - loss: 0.3068 - accuracy: 0.9054 - val_loss: 0.2134 - val_accuracy: 0.9347
Epoch 110/256
781/781 [==============================] - 5s 7ms/step - loss: 0.3126 - accuracy: 0.9034 - val_loss: 0.2118 - val_accuracy: 0.9350
Epoch 111/256
781/781 [==============================] - 5s 7ms/step - loss: 0.3121 - accuracy: 0.9037 - val_loss: 0.2110 - val_accuracy: 0.9351
Epoch 112/256
781/781 [==============================] - 5s 7ms/step - loss: 0.3046 - accuracy: 0.9059 - val_loss: 0.2104 - val_ac

781/781 [==============================] - 5s 7ms/step - loss: 0.2534 - accuracy: 0.9191 - val_loss: 0.1774 - val_accuracy: 0.9438
Epoch 163/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2527 - accuracy: 0.9177 - val_loss: 0.1769 - val_accuracy: 0.9438
Epoch 164/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2557 - accuracy: 0.9191 - val_loss: 0.1753 - val_accuracy: 0.9447
Epoch 165/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2508 - accuracy: 0.9200 - val_loss: 0.1757 - val_accuracy: 0.9441
Epoch 166/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2497 - accuracy: 0.9204 - val_loss: 0.1755 - val_accuracy: 0.9446
Epoch 167/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2501 - accuracy: 0.9195 - val_loss: 0.1744 - val_accuracy: 0.9447
Epoch 168/256
781/781 [==============================] - 6s 7ms/step - loss: 0.2462 - accuracy: 0.9206 - val_loss: 0.1745 - val_accuracy: 0.9447

781/781 [==============================] - 5s 7ms/step - loss: 0.2231 - accuracy: 0.9276 - val_loss: 0.1568 - val_accuracy: 0.9495
Epoch 219/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2209 - accuracy: 0.9280 - val_loss: 0.1563 - val_accuracy: 0.9494
Epoch 220/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2180 - accuracy: 0.9298 - val_loss: 0.1563 - val_accuracy: 0.9497
Epoch 221/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2178 - accuracy: 0.9283 - val_loss: 0.1554 - val_accuracy: 0.9494
Epoch 222/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2160 - accuracy: 0.9296 - val_loss: 0.1556 - val_accuracy: 0.9497
Epoch 223/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2158 - accuracy: 0.9302 - val_loss: 0.1559 - val_accuracy: 0.9496
Epoch 224/256
781/781 [==============================] - 5s 7ms/step - loss: 0.2179 - accuracy: 0.9289 - val_loss: 0.1547 - val_accuracy: 0.9501

epocas --> certeza

128 --> 0.91  
165 --> 0.92  
230 --> 0.93  
256 --> 0.933